# Phase 1: Core Infrastructure

Build the base platform that all strategies will run on. This notebook covers:

- **Exchange connectivity** — REST/WebSocket patterns for market data
- **Market data pipeline** — Price feeds, OHLCV candles, order books
- **Order execution** — Market, limit, and stop orders
- **Basic margin trading** — Leverage mechanics, buying power, margin requirements
- **Position tracking** — P&L calculation, portfolio state

---

## Prerequisites

```bash
pip install pandas numpy matplotlib
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from enum import Enum
from typing import Optional
import json
from datetime import datetime, timedelta

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

---
## 1.1 Simulated Market Data Pipeline

Before connecting to real exchanges, we need to understand the shape of market data.
Here we generate synthetic OHLCV (Open, High, Low, Close, Volume) candles using
geometric Brownian motion — the standard model for stock price simulation.

### Key parameters to experiment with:
- `mu` (drift) — expected annual return
- `sigma` (volatility) — annual standard deviation of returns
- `dt` — time step (1/252 for daily, 1/(252*24) for hourly)

In [ ]:
def generate_ohlcv(
    days: int = 252,
    initial_price: float = 100.0,
    mu: float = 0.08,        # 8% annual drift — try changing this!
    sigma: float = 0.20,     # 20% annual volatility — try changing this!
    candles_per_day: int = 1  # 1 = daily, 24 = hourly
) -> pd.DataFrame:
    """Generate synthetic OHLCV data using geometric Brownian motion."""
    n = days * candles_per_day
    dt = 1.0 / (252 * candles_per_day)

    # GBM: dS = S * (mu*dt + sigma*sqrt(dt)*Z)
    returns = np.random.normal(mu * dt, sigma * np.sqrt(dt), n)
    prices = initial_price * np.exp(np.cumsum(returns))

    # Build OHLCV from close prices
    dates = pd.date_range('2024-01-01', periods=n, freq=f'{24 // candles_per_day}h')
    intraday_noise = sigma * np.sqrt(dt) * 0.5

    df = pd.DataFrame({
        'timestamp': dates,
        'open':   prices * (1 + np.random.normal(0, intraday_noise, n)),
        'high':   prices * (1 + np.abs(np.random.normal(0, intraday_noise, n))),
        'low':    prices * (1 - np.abs(np.random.normal(0, intraday_noise, n))),
        'close':  prices,
        'volume': np.random.lognormal(mean=10, sigma=1, size=n).astype(int)
    })
    df.set_index('timestamp', inplace=True)
    return df


# Generate 1 year of daily candles
market_data = generate_ohlcv(days=252)
print(f"Generated {len(market_data)} candles")
print(f"Price range: ${market_data['low'].min():.2f} — ${market_data['high'].max():.2f}")
market_data.head(10)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), gridspec_kw={'height_ratios': [3, 1]})

ax1.plot(market_data.index, market_data['close'], color='steelblue', linewidth=1.2)
ax1.fill_between(market_data.index, market_data['low'], market_data['high'], alpha=0.15, color='steelblue')
ax1.set_title('Simulated Price Data (GBM)', fontsize=14)
ax1.set_ylabel('Price ($)')

ax2.bar(market_data.index, market_data['volume'], color='gray', alpha=0.6, width=0.8)
ax2.set_ylabel('Volume')
ax2.set_xlabel('Date')

plt.tight_layout()
plt.show()

### Exercise 1.1

Try generating data with different parameters:
1. High volatility crypto-like asset: `sigma=0.80, mu=0.15`
2. Low volatility bond-like asset: `sigma=0.05, mu=0.03`
3. Hourly candles: `candles_per_day=24`

What do you notice about the high/low spread as volatility increases?

In [ ]:
# YOUR CODE HERE — generate different market data and plot it
# crypto_data = generate_ohlcv(sigma=0.80, mu=0.15)
# bond_data = generate_ohlcv(sigma=0.05, mu=0.03)


---
## 1.2 Order Types & Execution Simulation

A trading bot needs to place different types of orders:

| Order Type | Description | When to Use |
|---|---|---|
| **Market** | Execute immediately at current price | Urgent entries/exits |
| **Limit** | Execute only at specified price or better | Price-sensitive entries |
| **Stop** | Trigger market order when price hits level | Stop-losses |
| **Stop-Limit** | Trigger limit order when price hits level | Precise stop-losses |

In [ ]:
class OrderType(Enum):
    MARKET = 'market'
    LIMIT = 'limit'
    STOP = 'stop'
    STOP_LIMIT = 'stop_limit'

class Side(Enum):
    BUY = 'buy'
    SELL = 'sell'

class OrderStatus(Enum):
    PENDING = 'pending'
    FILLED = 'filled'
    CANCELLED = 'cancelled'
    REJECTED = 'rejected'

@dataclass
class Order:
    order_id: str
    symbol: str
    side: Side
    order_type: OrderType
    quantity: float
    price: Optional[float] = None       # for limit/stop-limit
    stop_price: Optional[float] = None  # for stop/stop-limit
    status: OrderStatus = OrderStatus.PENDING
    filled_price: Optional[float] = None
    filled_at: Optional[str] = None
    slippage: float = 0.0


class OrderSimulator:
    """Simulates order execution against OHLCV data."""

    def __init__(self, slippage_bps: float = 5.0, fee_bps: float = 10.0):
        self.slippage_bps = slippage_bps  # basis points of slippage
        self.fee_bps = fee_bps            # basis points per trade
        self.order_count = 0

    def submit_order(self, symbol: str, side: Side, order_type: OrderType,
                     quantity: float, price: float = None,
                     stop_price: float = None) -> Order:
        self.order_count += 1
        return Order(
            order_id=f"ORD-{self.order_count:04d}",
            symbol=symbol, side=side, order_type=order_type,
            quantity=quantity, price=price, stop_price=stop_price
        )

    def try_fill(self, order: Order, candle: pd.Series) -> Order:
        """Attempt to fill an order given a candle's OHLCV data."""
        if order.status != OrderStatus.PENDING:
            return order

        fill_price = None

        if order.order_type == OrderType.MARKET:
            fill_price = candle['open']  # markets fill at open of next candle

        elif order.order_type == OrderType.LIMIT:
            if order.side == Side.BUY and candle['low'] <= order.price:
                fill_price = min(order.price, candle['open'])
            elif order.side == Side.SELL and candle['high'] >= order.price:
                fill_price = max(order.price, candle['open'])

        elif order.order_type == OrderType.STOP:
            if order.side == Side.SELL and candle['low'] <= order.stop_price:
                fill_price = order.stop_price  # simplified
            elif order.side == Side.BUY and candle['high'] >= order.stop_price:
                fill_price = order.stop_price

        if fill_price is not None:
            # Apply slippage
            slippage_mult = 1 + (self.slippage_bps / 10000)
            if order.side == Side.BUY:
                fill_price *= slippage_mult
            else:
                fill_price /= slippage_mult

            order.filled_price = round(fill_price, 4)
            order.slippage = abs(fill_price - (order.price or candle['open'])) / (order.price or candle['open']) * 10000
            order.status = OrderStatus.FILLED
            order.filled_at = str(candle.name)

        return order

    def calculate_fee(self, order: Order) -> float:
        """Calculate trading fee for a filled order."""
        if order.status != OrderStatus.FILLED:
            return 0.0
        return order.filled_price * order.quantity * (self.fee_bps / 10000)


# Demo: place and fill different order types
sim = OrderSimulator(slippage_bps=5, fee_bps=10)

candle = market_data.iloc[50]
print(f"Candle: O={candle['open']:.2f} H={candle['high']:.2f} L={candle['low']:.2f} C={candle['close']:.2f}")
print()

# Market order
mkt = sim.submit_order('SYN/USD', Side.BUY, OrderType.MARKET, quantity=10)
mkt = sim.try_fill(mkt, candle)
print(f"Market BUY: filled at ${mkt.filled_price:.2f}, fee=${sim.calculate_fee(mkt):.2f}")

# Limit order below current price
lmt = sim.submit_order('SYN/USD', Side.BUY, OrderType.LIMIT, quantity=10, price=candle['low'] + 0.5)
lmt = sim.try_fill(lmt, candle)
print(f"Limit BUY @${lmt.price:.2f}: {'filled at $' + f'{lmt.filled_price:.2f}' if lmt.status == OrderStatus.FILLED else 'NOT FILLED'}")

# Stop order
stp = sim.submit_order('SYN/USD', Side.SELL, OrderType.STOP, quantity=10, stop_price=candle['low'] + 0.1)
stp = sim.try_fill(stp, candle)
print(f"Stop SELL @${stp.stop_price:.2f}: {'filled at $' + f'{stp.filled_price:.2f}' if stp.status == OrderStatus.FILLED else 'NOT FILLED'}")

### Exercise 1.2

1. Change `slippage_bps` to 0, 10, 50 — how does it affect fill prices?
2. Place a limit sell order above the high — does it fill? Why or why not?
3. How much does a 10 bps fee cost on a $10,000 position? Calculate it manually, then verify with the simulator.

In [ ]:
# YOUR CODE HERE


---
## 1.3 Leverage & Margin Mechanics

Leverage lets you control a large position with a small amount of capital.

**Key formulas:**
- `Position Value = Quantity × Price`
- `Margin Required = Position Value / Leverage`
- `Buying Power = Equity × Leverage`
- `Margin Utilization = Used Margin / Equity`
- `Liquidation Price (long) = Entry × (1 - 1/Leverage + Maintenance Margin Rate)`

In [ ]:
@dataclass
class MarginAccount:
    equity: float                        # your actual capital
    leverage: float = 10.0               # max leverage allowed
    maintenance_margin_rate: float = 0.05 # 5% maintenance margin
    positions: dict = field(default_factory=dict)

    @property
    def buying_power(self) -> float:
        return self.equity * self.leverage

    @property
    def used_margin(self) -> float:
        return sum(abs(p['value']) / self.leverage for p in self.positions.values())

    @property
    def free_margin(self) -> float:
        return self.equity - self.used_margin + self.unrealized_pnl

    @property
    def margin_utilization(self) -> float:
        if self.equity == 0:
            return float('inf')
        return self.used_margin / self.equity

    @property
    def unrealized_pnl(self) -> float:
        return sum(p.get('unrealized_pnl', 0) for p in self.positions.values())

    def open_position(self, symbol: str, side: str, quantity: float, price: float) -> dict:
        position_value = quantity * price
        margin_needed = position_value / self.leverage

        if margin_needed > self.free_margin:
            return {'error': f'Insufficient margin. Need ${margin_needed:.2f}, have ${self.free_margin:.2f}'}

        direction = 1 if side == 'long' else -1
        self.positions[symbol] = {
            'side': side,
            'quantity': quantity,
            'entry_price': price,
            'value': position_value * direction,
            'margin': margin_needed,
            'unrealized_pnl': 0.0
        }
        return self.positions[symbol]

    def update_price(self, symbol: str, current_price: float):
        if symbol not in self.positions:
            return
        pos = self.positions[symbol]
        direction = 1 if pos['side'] == 'long' else -1
        pos['unrealized_pnl'] = direction * pos['quantity'] * (current_price - pos['entry_price'])
        pos['current_price'] = current_price

    def liquidation_price(self, symbol: str) -> float:
        pos = self.positions.get(symbol)
        if not pos:
            return 0.0
        entry = pos['entry_price']
        if pos['side'] == 'long':
            return entry * (1 - 1/self.leverage + self.maintenance_margin_rate)
        else:
            return entry * (1 + 1/self.leverage - self.maintenance_margin_rate)

    def summary(self) -> str:
        lines = [
            f"Equity:        ${self.equity:>12,.2f}",
            f"Leverage:      {self.leverage:>12.1f}x",
            f"Buying Power:  ${self.buying_power:>12,.2f}",
            f"Used Margin:   ${self.used_margin:>12,.2f}",
            f"Free Margin:   ${self.free_margin:>12,.2f}",
            f"Margin Util:   {self.margin_utilization:>11.1%}",
            f"Unrealized PnL:${self.unrealized_pnl:>12,.2f}",
        ]
        return '\n'.join(lines)


# Demo: open a leveraged position
account = MarginAccount(equity=10_000, leverage=10)
print("=== Account Before Trade ===")
print(account.summary())
print()

# Open a 10x leveraged long position
entry_price = market_data['close'].iloc[0]
position_size = account.buying_power * 0.5 / entry_price  # use 50% of buying power
result = account.open_position('SYN/USD', 'long', position_size, entry_price)
print(f"Opened LONG {position_size:.2f} SYN/USD @ ${entry_price:.2f}")
print(f"Position Value: ${position_size * entry_price:,.2f}")
print(f"Liquidation Price: ${account.liquidation_price('SYN/USD'):.2f}")
print()

print("=== Account After Trade ===")
print(account.summary())

In [ ]:
# Simulate price movement and track P&L
pnl_history = []
equity_history = []
margin_util_history = []

for i, (ts, candle) in enumerate(market_data.iterrows()):
    account.update_price('SYN/USD', candle['close'])
    pnl_history.append(account.unrealized_pnl)
    equity_history.append(account.equity + account.unrealized_pnl)
    margin_util_history.append(account.margin_utilization)

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

axes[0].plot(market_data.index, market_data['close'], color='steelblue')
axes[0].axhline(y=entry_price, color='green', linestyle='--', alpha=0.7, label=f'Entry ${entry_price:.2f}')
axes[0].axhline(y=account.liquidation_price('SYN/USD'), color='red', linestyle='--', alpha=0.7, label=f'Liquidation ${account.liquidation_price("SYN/USD"):.2f}')
axes[0].set_ylabel('Price ($)')
axes[0].set_title('Price, P&L, and Equity with 10x Leverage')
axes[0].legend()

colors = ['green' if x >= 0 else 'red' for x in pnl_history]
axes[1].fill_between(market_data.index, pnl_history, 0, where=[p >= 0 for p in pnl_history], color='green', alpha=0.3)
axes[1].fill_between(market_data.index, pnl_history, 0, where=[p < 0 for p in pnl_history], color='red', alpha=0.3)
axes[1].plot(market_data.index, pnl_history, color='black', linewidth=0.5)
axes[1].set_ylabel('Unrealized P&L ($)')
axes[1].axhline(y=0, color='gray', linewidth=0.5)

axes[2].plot(market_data.index, equity_history, color='purple')
axes[2].axhline(y=10_000, color='gray', linestyle='--', alpha=0.5, label='Starting Equity')
axes[2].set_ylabel('Total Equity ($)')
axes[2].set_xlabel('Date')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"\nFinal equity: ${equity_history[-1]:,.2f} (started at $10,000)")
print(f"Return: {(equity_history[-1] - 10_000) / 10_000:.1%}")
print(f"Max drawdown: ${min(pnl_history):,.2f}")
print(f"Max gain: ${max(pnl_history):,.2f}")

---
## 1.4 Leverage Comparison

How does leverage amplify both gains AND losses? Let's compare 1x, 2x, 5x, 10x, and 25x leverage on the same price data.

In [ ]:
leverage_levels = [1, 2, 5, 10, 25]
equity_start = 10_000
entry = market_data['close'].iloc[0]

fig, ax = plt.subplots(figsize=(14, 7))

results = {}
for lev in leverage_levels:
    position_value = equity_start * lev
    qty = position_value / entry
    returns_pct = (market_data['close'] - entry) / entry
    equity_curve = equity_start + (returns_pct * position_value)
    # Clip at zero (liquidation)
    equity_curve = equity_curve.clip(lower=0)
    ax.plot(market_data.index, equity_curve, label=f'{lev}x leverage', linewidth=1.5)
    results[lev] = {
        'final': equity_curve.iloc[-1],
        'max': equity_curve.max(),
        'min': equity_curve.min(),
        'liquidated': (equity_curve == 0).any()
    }

ax.axhline(y=equity_start, color='gray', linestyle='--', alpha=0.5)
ax.axhline(y=0, color='red', linewidth=2, alpha=0.3, label='Liquidation')
ax.set_title('Equity Curves at Different Leverage Levels', fontsize=14)
ax.set_ylabel('Account Equity ($)')
ax.set_xlabel('Date')
ax.legend()
plt.tight_layout()
plt.show()

# Summary table
print(f"{'Leverage':>10} {'Final Equity':>15} {'Return':>10} {'Max Equity':>15} {'Min Equity':>15} {'Liquidated':>12}")
print('-' * 80)
for lev, r in results.items():
    ret = (r['final'] - equity_start) / equity_start
    print(f"{lev:>9}x ${r['final']:>14,.2f} {ret:>9.1%} ${r['max']:>14,.2f} ${r['min']:>14,.2f} {'YES' if r['liquidated'] else 'no':>12}")

### Exercise 1.3

1. Regenerate `market_data` with `sigma=0.50` (high volatility). Which leverage levels get liquidated?
2. At 10x leverage, what percentage price drop causes liquidation? Calculate mathematically, then verify.
3. Try adjusting `maintenance_margin_rate` in the MarginAccount. How does 2% vs 10% affect the liquidation price?

In [ ]:
# YOUR CODE HERE


---
## 1.5 Position Tracking & P&L

A proper position tracker manages multiple positions, calculates realized and unrealized P&L, and maintains an audit trail.

In [ ]:
@dataclass
class Trade:
    symbol: str
    side: str
    quantity: float
    entry_price: float
    exit_price: float
    entry_time: str
    exit_time: str
    pnl: float
    return_pct: float
    fees: float


class PositionTracker:
    def __init__(self, fee_bps: float = 10.0):
        self.fee_bps = fee_bps
        self.positions = {}
        self.closed_trades: list[Trade] = []

    def open(self, symbol: str, side: str, qty: float, price: float, timestamp: str):
        self.positions[symbol] = {
            'side': side, 'quantity': qty,
            'entry_price': price, 'entry_time': timestamp,
            'current_price': price, 'unrealized_pnl': 0.0
        }

    def close(self, symbol: str, price: float, timestamp: str) -> Optional[Trade]:
        pos = self.positions.pop(symbol, None)
        if not pos:
            return None
        direction = 1 if pos['side'] == 'long' else -1
        gross_pnl = direction * pos['quantity'] * (price - pos['entry_price'])
        entry_fee = pos['quantity'] * pos['entry_price'] * (self.fee_bps / 10000)
        exit_fee = pos['quantity'] * price * (self.fee_bps / 10000)
        net_pnl = gross_pnl - entry_fee - exit_fee
        ret_pct = net_pnl / (pos['quantity'] * pos['entry_price'])

        trade = Trade(
            symbol=symbol, side=pos['side'], quantity=pos['quantity'],
            entry_price=pos['entry_price'], exit_price=price,
            entry_time=pos['entry_time'], exit_time=timestamp,
            pnl=net_pnl, return_pct=ret_pct, fees=entry_fee + exit_fee
        )
        self.closed_trades.append(trade)
        return trade

    def update(self, symbol: str, price: float):
        pos = self.positions.get(symbol)
        if not pos:
            return
        direction = 1 if pos['side'] == 'long' else -1
        pos['current_price'] = price
        pos['unrealized_pnl'] = direction * pos['quantity'] * (price - pos['entry_price'])

    def trade_log(self) -> pd.DataFrame:
        if not self.closed_trades:
            return pd.DataFrame()
        return pd.DataFrame([vars(t) for t in self.closed_trades])


# Simulate a simple strategy: buy and hold for 20 candles, repeat
tracker = PositionTracker(fee_bps=10)
hold_period = 20  # candles

i = 0
while i < len(market_data) - hold_period:
    entry_candle = market_data.iloc[i]
    exit_candle = market_data.iloc[i + hold_period]

    tracker.open('SYN/USD', 'long', 100, entry_candle['close'], str(market_data.index[i]))
    trade = tracker.close('SYN/USD', exit_candle['close'], str(market_data.index[i + hold_period]))
    i += hold_period

trade_log = tracker.trade_log()
print(f"Total trades: {len(trade_log)}")
print(f"Total P&L: ${trade_log['pnl'].sum():,.2f}")
print(f"Win rate: {(trade_log['pnl'] > 0).mean():.1%}")
print(f"Total fees: ${trade_log['fees'].sum():,.2f}")
print(f"Avg return/trade: {trade_log['return_pct'].mean():.2%}")
print()
trade_log[['side', 'quantity', 'entry_price', 'exit_price', 'pnl', 'return_pct', 'fees']].round(4)

### Exercise 1.4

1. Change `hold_period` to 5, 10, 50 — how does trade frequency affect total fees vs total P&L?
2. Add a short-selling strategy: sell when the 10-day return is positive, buy when negative.
3. Load your own data (CSV with date, open, high, low, close, volume columns) and run the position tracker on it.

```python
# Example: load your own data
# my_data = pd.read_csv('my_prices.csv', parse_dates=['date'], index_col='date')
```

In [ ]:
# YOUR CODE HERE


---
## 1.6 Comprehension Check

Answer these questions (write your answers in the cells below):

1. With $5,000 equity and 20x leverage, what is your buying power?
2. If you go long 50 units at $200 with 10x leverage, how much margin is used?
3. At 10x leverage with a 5% maintenance margin rate, what price drop (%) triggers liquidation on a long?
4. You make 10 trades with 10 bps fees each, each with $50,000 notional. What are your total fees?
5. Why does higher trading frequency increase the drag of fees on returns?

In [ ]:
# YOUR ANSWERS / CALCULATIONS HERE
# Q1: buying_power = ?
# Q2: margin_used = ?
# Q3: liquidation_drop = ?
# Q4: total_fees = ?
# Q5: (explain in a comment)
